# Addition with Demonstration and GPT-Neo

Parts of this Colab notebook are based on "[Train a GPT-2 (or GPT Neo) Text-Generating Model w/ GPU](https://colab.research.google.com/drive/15qBZx5y9rdaQSyWpsreMDnTiZ5IlN0zD?usp=sharing)" by [Max Woolf](https://minimaxir.com) and utilize Max's [aitextgen package](https://github.com/minimaxir/aitextgen).


In [ ]:
!pip uninstall -qqy torch torchvision torchtext torchaudio torchdata fastai
!pip install -qq torch==1.11.0 pytorch-lightning==1.7.7 aitextgen==0.6.0

In [13]:
import numpy as np
import shutil
import math
import re
import datetime
import os
import random
import sys
from google.colab import files
from collections import defaultdict
from copy import copy

model_name = "EleutherAI/gpt-neo-125M"
MAX_LENGTH = 10000  # Maximum length of output from GPT Neo, in # of tokens

## Downloading GPT Neo

Download a GPT Neo model and load it into the GPU. 

In [2]:
import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

ai = aitextgen(model=model_name, to_gpu=True)

INFO:aitextgen:Loading EleutherAI/gpt-neo-125M model from /aitextgen.
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.1"
}

INFO:aitextgen:Using the tokenizer for EleutherAI/gpt-neo-125M.
loading file vocab.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/vocab.json
loading file merges.txt from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/merges.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/special_tokens_map.json
loading file tokenizer_config.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/b983397156c0991016feccfbcbe1fe2746d47b29/tokenizer_config.j

## Mounting Google Drive

Running this cell will mount your Google Drive in Colab in order to save and load datasets and model checkpoints.

In [3]:
mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Specify the training file and the validation or test file

The following code expects the training file and the validation or test file to be located within a folder in your Google Drive called "Colab Inputs".

In [4]:
training_file_name = 'train_dataset.txt'
validation_or_test_file_name = 'val_dataset.txt'
unbalanced_file_name = 'val_dataset_unbalanced.txt'
input_path = "/content/drive/My Drive/Colab Inputs/"

shutil.copyfile(input_path + training_file_name, training_file_name)
shutil.copyfile(input_path + validation_or_test_file_name, validation_or_test_file_name)
shutil.copyfile(input_path + unbalanced_file_name, unbalanced_file_name)

'val_dataset_unbalanced.txt'

## Finetuning (Skip this if loading a trained model)

Note - **Skip this cell** if you instead plan to load a trained model.

<br/>

This cell will start the actual finetuning of GPT-Neo in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.


In [ ]:
!git clone https://github.com/NVIDIA/apex
%pip install -v --no-cache-dir ./apex

In [6]:
ai.train(training_file_name,
         line_by_line=False,
         from_cache=False,
         num_steps=10000,
         generate_every=20000,
         save_every=2500,
         save_gdrive=True,
         learning_rate=0.00005,
         batch_size=1,
         fp16=True,
         )

INFO:aitextgen:Loading text from train_dataset.txt with generation length of 2048.


  0%|          | 0/200 [00:00<?, ?it/s]

INFO:aitextgen.TokenDataset:Encoding 200 sets of tokens from train_dataset.txt.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit apex Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:258: LightningDeprecationWarning: The `Callback.on_batch_end` hook was deprecated in v1.6 and will be removed i

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/usr/local/lib/python3.10/dist-packages/apex/__init__.py:68: DeprecatedFeatureWarning: apex.amp is deprecated and will be removed by the end of February 2023. Use [PyTorch AMP](https://pytorch.org/docs/stable/amp.html)
  warnings.warn(msg, DeprecatedFeatureWarning)


...

Configuration saved in trained_model/generation_config.json


7,500 steps reached: saving model to /trained_model
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0


Configuration saved in trained_model/generation_config.json


10,000 steps reached: saving model to /trained_model


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=10000` reached.
INFO:aitextgen:Saving trained model pytorch_model.bin to /trained_model
Configuration saved in trained_model/generation_config.json



## Load a Trained Model (can skip if you finetuned a model just now using the cell above)

Note - **This cell can be skipped** if you just finetuned a model using the cell above during this same session. However, if you restart the runtime and wish to use a model you've finetuned, you'll want to follow the instructions below.

<br/>


If you already had a trained model from this notebook, set `from_folder` to the name of the folder that contains it; running the next cell will copy the `pytorch_model.bin` and the `config.json` file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive).

In [10]:
from_folder = 'ATG_20230510_221524'

for file in ['pytorch_model.bin', 'config.json']:
  if from_folder:
    copy_file_from_gdrive(file, from_folder)
  else:
    copy_file_from_gdrive(file)

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text.

**If you just trained a model** during this same session, you may choose to run the next cell to reload the model you just trained from the `trained_model` folder. You may get faster performance this way.

In [7]:
ai = aitextgen(model_folder="trained_model", to_gpu=True)

INFO:aitextgen:Loading model from provided weights and config in /trained_model.
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.1"
}

loading configuration file trained_model/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.1"
}

INFO:aitextgen:GPTNeo loaded with 125M parameters.
INFO:aitextgen:Using the default GPT-2 Tokenizer.


## Code for environment forcing and evaluation

At inference time, a model can theoretically generate a symbol sequence such as “write *coord1* 5 look *coord1*  8”, even though interpreting the model’s generations as a sequence of actions would incorrectly imply that one would “see” the symbol 5 rather than 8 when looking at *coord1*.  To address this, a separate process interprets model output as a sequence of actions as it is generated. The initial intent was for this process to halt generation immediately whenever the model produced a string of the form “look *coord1”*, append the symbol currently at coord1, and restart generation with this amended output used as the prefix. In practice, it's most efficient only to do this only in the rare cases when the symbol generated after “look *coord1” differs from the symbol that actually appears at the corresponding coordinate when the output is interpreted as actions. We refer to this approach as 'environment forcing' by analogy to related concepts such as professor forcing and memory forcing, as it allows the state of the environment to impose constraints on the generated text that may mitigate exposure bias. It also allows information to be stored in the environment and later retrieved even if this information were to fall outside of the context window .

In [20]:
OUTPUT_PATH = '/content/drive/My Drive/Colab Output/'

# Generate this if we have exceeded maximum environment forcing failures
TOO_MANY_SURPRISES = 'Too many surprises'   
# Maximum number of environment forcing failures before we give up
MAX_SURPRISES = 30  
# Maximum number of times we should try generating additional text when we have 
# exceeded GPT-Neo's max output and have not yet generated text of the form 
# '{ result is ### }'
MAX_TRIES = 4  
# When trying to generate additional text, how many tokens from the previous 
# output do we use as the prefix?
MAX_TOKENS = 2048  

# tokens that indicate the beginning of an action
ACTION_TOKENS = { 'look', 'write', '{', 'stop', 'clear' }  
# Symbol representing location we are currently looking at when visualizing the grid
LOOK_SYMBOL = '@' 
# Symbol representing location we are currently writing something at when visualizing the grid
WRITE_SYMBOL = '*' 
# Symbol representing location we are currently looking at and writing to simultaneously
# (shouldn't be possible, but want to notice if it happens)
LOOK_AND_WRITE_SYMBOL = '!' 

def write_output(s, output_file):
    """
    Write `s` to a file on its own line and flush the file

    :param str s: Input string
    :param file output_file: File to write to
    :return: None
    """
    print(s)
    if output_file is not None:
      output_file.write(s + '\n')
      output_file.flush()


def get_result(text):
    """
    Extract the string 'result is \d+' from a string that may contain it

    :param str text: string that may contain result
    :return: 'result is \d+' or '(no answer)'
    """
    m = re.search(r'result is (.*) }', text)
    if m:
      return re.sub(r'(?<!\d)\d{3}(?!\d)', '', m.group(1)).replace(' ', '')
    else:
      return '(no answer)'


def default_pos2cell(pos):
    """
    Default function for converting a string encoding of some coordinates to 
    the x-y coordinate it represents

    :param str pos: string encoding of coordinates in form \d\d,\d\d:\d\d\d
    :return: (x, y) coordinate tuple
    """
    def to_cell(s):
      if s == '00':
        return 0
      else:
        num_str = s.lstrip('0')
        if num_str.isnumeric():
          return int(num_str)
        else:
          return -1

    pos = re.sub(':\d\d\d$', '', pos)
    tokens = pos.partition(',')
    row = to_cell(tokens[0])
    col = to_cell(tokens[2])
    return (row, col)


class Environment:
  """
  Represents the 'environment' - the 'piece of paper' we are writing to and looking at.

  Attributes:
  grid: defaultdict, key: coordinate tuple; val: strings representing what is
        at each coordinate of the piece of paper
  looking_at: defaultdict, key: tuple indicating where we were looking during
              the last looking action; val: what we expected to see there
  pen_at: list of tuples indicating where our pen went during the last writing
          action
  """

  def __init__(self):
      """ Initialize the environment """
      self.grid = defaultdict(lambda:'_')
      self.looking_at = defaultdict(lambda:'_')
      self.pen_at = []

  def _grid_as_string(self, cell_condition_fn):
      """ 
      Return a string representation of the grid which includes all cells 
      where cell_condition_fn is true 
      """

      minimum_x = sys.maxsize
      minimum_y = sys.maxsize
      maximum_x = 0
      maximum_y = 0
      is_empty = True
      for cell in self.grid:
          # we're not worried about cells not meeting the cell_condition_fn
          if cell_condition_fn(cell):   
              if cell[0] < minimum_x:
                  minimum_x = cell[0]
              if cell[1] < minimum_y:
                  minimum_y = cell[1]
              if cell[0] > maximum_x:
                  maximum_x = cell[0]
              if cell[1] > maximum_y:
                  maximum_y = cell[1]
              is_empty = False

      # Return empty string if no cells meeting cell_condition_fn
      if is_empty:
          return ''

      s = '\n'
      for y in range(minimum_y, maximum_y + 1):
          for x in range(minimum_x, maximum_x + 1):
              mod_char = ' '
              if (x,y) in self.looking_at:
                  mod_char = LOOK_SYMBOL
              if (x,y) in self.pen_at:
                  mod_char = WRITE_SYMBOL
                  # The following shouldn't be possible the way we currently 
                  # have it set up, but want to notice if it is
              if ((x,y) in self.looking_at) and ((x,y) in self.pen_at): 
                  mod_char = LOOK_AND_WRITE_SYMBOL
              s += mod_char + self.grid[(x, y)]
          s += '\n'
      return s + '\n'

  def division_grid_as_string(self):
      """
      Return a string representation of the part of the grid used for division
      (to the left of column 60)
      """
      # we're not worried about cells to the right of x == 60,
      # we consider those part of the 'addition grid'
      return self._grid_as_string(lambda cell: cell[0] < 60)

  def addition_grid_as_string(self):
      """
      Return a string representation of the part of the grid used for addition
      (to the right of & including column 60)
      """
      # we're not worried about cells at or to the left of x == 60,
      # we consider those part of the 'addition grid'
      return self._grid_as_string(lambda cell: cell[0] >= 60)

  def _stringify_interesting_bits_of_grid_only(self):
      """
      Return a string representation of any parts of the grid that
      are being looked at or written in
      """
      def was_acted_upon(s):
          return (LOOK_SYMBOL in s) or (WRITE_SYMBOL in s) or (LOOK_AND_WRITE_SYMBOL in s)

      s1 = self.division_grid_as_string()
      s2 = self.addition_grid_as_string()
      if was_acted_upon(s1):
          if was_acted_upon(s2):
              return s1 + '\n--\n' + s2
          else:
              return s1
      elif was_acted_upon(s2):
          return s2
      else:
          return ''

  def __repr__(self):
      """
      Return a string representation of any parts of the grid that
      are being looked at or written in
      """
      return self._stringify_interesting_bits_of_grid_only()


class Generation:
    """
    Represents text generated by GPT-Neo (after the prompt), in the process of 
    being interpreted in order to execute the actions within it.

    Attributes:
    tokens: List of tokens to be interpreted
    action: Last action extracted by our interpreter; list of tokens starting
            with an action token
    index: Where we are in this list; we've already processed tokens before
           index, and are currently processing self.tokens[self.index]
    success: Whether the last attempt of the interpreter succeeded.
    done: True if we have just consumed a STOP instruction.

    """

    def __init__(self, tokens):
        """ Initialize an empty Generation object """
        self.tokens = tokens
        self.action = []
        self.index = 0
        self.success = None
        self.done = False

    def finish(self):
        """ Do final clean-up when we have read everything we are going to read from a generation """
        self.done = True
        self.action = ['stop']
        self.success = True

    def consume_action(self):
        """
        Consume the next action off of the generation object,
        update the current action to that action.
        """

        # Setup
        def is_end_token(t):
            return (t.startswith('\n') or t.startswith('í') or t.startswith('ì'))
        self.action = []
        self.success = True

        if self.index >= len(self.tokens):
            self.finish()
        else:
            word = self.tokens[self.index]
            if (word == ''):
                raise Error("Didn't expect to see empty tokens")

            if is_end_token(word):
                self.finish()

            elif word in ACTION_TOKENS:
                while True:
                    self.action.append(word)

                    # If this needs to be the last token of the action, then break:
                    if (self.index == len(self.tokens) - 1 or 
                        self.tokens[self.index + 1] in ACTION_TOKENS or
                        is_end_token(self.tokens[self.index + 1])):
                        self.index += 1
                        break
                    # otherwise, keep going:
                    else:
                        self.index += 1
                        word = self.tokens[self.index]
            else:
                raise Exception('Expected to see an action token!')
        

class ActionResult:
    """
    Represents the result of an executed action.

    Attributes:
    action: Action just interpreted; a list of tokens beginning with 'look', 'write', '{', or 'stop'
    environment: Current state of the environment after executing 'action'.
    unexpected_data: If the action expected to get one thing from the environment (e.g. "look 04,02:201 2"), 
                     and got another (e.g., there was not a 2 at position 04,02), then that unexpected data
                     will be here. Otherwise, evaluates to False.
    pos2cell: Function for converting a string encoding of some coordinates to the x-y coordinate it represents;
              should be the function that was used to encode any coordinates that may appear in `action`
    success: Whether this was a legal action. If not, success will be False and the action will be ignored.
    """
    def __init__(self, action, environment, unexpected_data, pos2cell):
        self.action = action
        self.environment = environment
        self.unexpected_data = unexpected_data
        self.pos2cell = pos2cell
        self.success = True

    def execute_action(self, action):
        """ 
        Execute an action and update the environment accordingly.

        :param list action: List of tokens corresponding to an action
        """

        assert(action)  # Shouldn't be empty
        assert(action[0] in ACTION_TOKENS)  # First token should definitely represent an action
        self.success = True
        self.unexpected_data = ''
        self.action = action
        self.environment.looking_at = defaultdict(lambda:'_')
        self.environment.pen_at = []

        if action[0] == 'stop':
            if len(action) != 1: self.success = False; return

        elif action[0] == 'clear':
            if len(action) != 1: self.success = False; return
            for key in copy(self.environment.grid):
                if key[0] >= 60:
                    self.environment.grid.pop(key)

        elif action[0] == 'write':   # e.g. "write 00,02:201 1 01,02:202 4" : write '1' at 00,02, then write '4' at 01,02
            if (len(action) == 1) or (len(action) % 2 == 0): 
                self.success = False; return

            for i in range(1, len(action), 2):
                cell = self.pos2cell(action[i])
                char = action[i + 1]
                self.environment.pen_at.append(cell)
                self.environment.grid[cell] = char

        elif action[0] == 'look':   # e.g. "look 00,02:201 1 01,02:202 4" : look at 00,02, expect to see '1', then look at 01,02, expect to see '4'
            if (len(action) == 1) or (len(action) % 2 == 0): 
                self.success = False; return

            # Build up a string representing what we saw,
            # and a string representing what we expected to see.
            # If they differ, then record the string representing
            # what we saw as 'unexpected data'.
            expected = ''
            observed = ''
            for i in range(1, len(action), 2):
                cell = self.pos2cell(action[i])
                char_expected = action[i + 1]
                char_observed = self.environment.grid[cell]
                expected += ' ' + action[i] + ' ' + char_expected
                observed += ' ' + action[i] + ' ' + char_observed
                self.environment.looking_at[cell] = char_expected
            
            if expected != observed:
                self.unexpected_data = observed.lstrip()

        elif action[0] == '{':    # no-op - doesn't affect environment, skip it
            pass

        else:
            assert('Illegal action should have been caught by generation.consume_action()!')
            self.success = False


def generate(test_prefix):
  """
  Generate a string based on the given prefix

  :param string test_prefix: Prefix for conditional generation from GPT Neo
  :rtype: string 
  """
  output = ai.generate(
                temperature=0.7,  # Value we provide for temperature - shouldn't matter as we just take the top response
                do_sample=False,  # If False, the generated text will be the optimal prediction at each time, and therefore deterministic.
                prompt=test_prefix,
                seed=0,
                max_length=MAX_LENGTH,
                return_as_list=True,
                )
  return output[0]


def score_it(line, target, pos2cell, out_file, score, listen_to_environment, report_score):
    """
    Score whether the answer in `line` is correct or incorrect; if correct, add 1 to the running score.
    In order to determine this, this function interprets the contents of line as actions and runs them.
    If it can, it determines whether the answer at the end of the line (if there is one) is correct, 
    but terminates early if it encounters a case where the model 'looked' at a particular location on 
    the grid and then outputted a symbol that was not actually the symbol appearing at that location.
    If just we want this latter functionality and don't care about the score at this point in time, we can 
    obtain it by setting `report_score` to False.
    
    The incoming line has not yet been environment-forced, so if `listen_to_environment` is true, we will need to check whether
    the symbol generated after substrings of the form “look coord1” differs from the symbol that actually 
    appears at coord1 on the grid when the output is interpreted as actions. If it does differ, then we 
    return (generation, action_result); the caller should force the regeneration of the result of the
    after inserting the symbol that the look action actually 'observed'. Otherwise, return a 2-tuple where
    the first element is the running score.

    :param str line: line being scored
    :param str target: desired result substring (output of get_result)
    :param function pos2cell: Function for converting a string encoding of some coordinates to the x-y coordinate it represents;
              should be the function that was used to encode any coordinates that may appear in `line`
    :param file out_file: output file
    :param int score: score so far; we may return a 2-tuple where the first arg is the same score or score + 1
    :param bool listen_to_environment: whether to implement environment forcing
    :param bool report_score: whether to report the score to out_file
    :return: (int score, None) or (Generation generation, ActionResult action_result)
    """

    if '\n' in line:
      parts = line.partition('\n')
      line = parts[0].strip()
    if '|' in line:
      parts = line.partition('|')
      line = parts[2].strip()

    write_output(f'#### score_it (listen_to_environment={listen_to_environment} #####\n', out_file)


    # Initialize environment
    environment = Environment()
    generation = Generation(line.split())
    action_result = ActionResult([], environment, '', pos2cell)
    write_output('Text to score:\n' + line + '\n\n', out_file)

    # Execute actions in sequence
    while True:
        generation.consume_action()

        if not generation.success:
            write_output('uninterpretable action: "' + ' '.join(generation.action) + '"\n', out_file)
            break
        elif generation.done:
            write_output('done.\n', out_file)
            break

        action_result.execute_action(generation.action)

        if not action_result.success:
            write_output('unsuccessful/illegal action (ignored, out_file): ' + ' '.join(generation.action) + '\n', out_file)
        else:
            pass

        """
        if action_result.unexpected_data:
            write_output('was surprised to see "' + action_result.unexpected_data + '"\n', out_file)
            if listen_to_environment:
                return (generation, action_result)
            else:
                write_output('however, not environment-forcing at the moment (yet we were still surprised!?)\n', out_file)
        """
        
    write_output('\n\n', out_file)


    # Finally time to see if we got the desired answer
    if listen_to_environment:
      write_output('Made it to end of score_it, any "suprises" should have been corrected by this point.', out_file)
    else:
      write_output('Made it to end of score_it, not environment-forcing so assuming we saw no surprises.', out_file)
    final_string = line.strip()
    answer = get_result(final_string)

    exactly_correct = (target == answer)
    if exactly_correct:
      score += 1

    if report_score:
      write_output(f"looking for answer within '{final_string}'...", out_file)
      write_output(f"{exactly_correct}! Model answer: {answer}. Target answer: {target}", out_file)

    return (score, None)


def generate_ef(test_prefix, target, pos2cell, out_file, score, front_end):
  """
  Generate language model output using the prefix `test_prefix`, but use environment forcing:
  if it encounters a case where the model 'looked' at a particular location on 
  the grid and then outputted a symbol that was not actually the symbol appearing at that location,
  it corrects the response to reflect the symbol the model actually 'saw' and re-generates the result
  of the string.
  
  :param str test_prefix: Prefix for conditional generation from GPT Neo
  :param str target: desired result substring (output of get_result)
  :param function pos2cell: Function for converting a string encoding of some coordinates to the x-y coordinate it represents;
            should be the function that was used to encode any coordinates that may appear in `line`
  :param file out_file: output file
  :param int score: score so far
  :param str front_end: any text that was previously generated in response to the original prefix provided in the test
                        or validation file. (test_prefix may be this original prefix, but it may not be, if the generated
                        text exceeded GPT Neo's max generation window size and it still has not generated a phrase of the form
                        '{ result is ### }'.
  :return: combined string consisting of front_end + the new string that was generated
  """

  write_output(f'############ generate_ef ############\n', out_file)
  line = generate(test_prefix)
  write_output(f'front_end: {front_end}', out_file)
  write_output(f'Prompt (len={len(test_prefix)})                     : {test_prefix}', out_file)
  write_output(f'Generation (includes prompt) (len={len(line)}): {line}', out_file)
  combined = (front_end.rstrip() + ' ' + line.lstrip()).strip()
  result = score_it(combined, target, pos2cell, out_file, score, listen_to_environment=False, report_score=False)
  
  # If we didn't get what we expected from the environment:
  surprises = 0
  while (type(result[0]) is Generation):
      surprises += 1
      if (surprises > MAX_SURPRISES):
        write_output(f"Too many surprises. Terminating early\n", out_file)
        return combined + ' ' + TOO_MANY_SURPRISES + '\n'  # \n so that the outer loop terminates

      generation = result[0]
      action_result = result[1]
      write_output(f"was surprised: action = '{' '.join(generation.action)}', unexpected_data='{action_result.unexpected_data}'", out_file)

      action_len = len(generation.action)
      lhs = generation.tokens[0:(generation.index - action_len + 1)]
      lhs.append(action_result.unexpected_data)
      lhs_string = ' '.join(lhs)
      write_output(f"Regenerating with prompt '{lhs_string}'", out_file)

      # Regenerate with new prompt
      tokens = lhs_string.split(' ')
      dividing_word = len(tokens) - MAX_TOKENS
      tail_end = ' '.join(tokens[dividing_word:])
      front_end = ' '.join(tokens[:dividing_word])
      line = generate(tail_end)

      # Re-score
      write_output(f"New prompt: '{tail_end}'", out_file)
      write_output(f"New generation: '{line}'", out_file)
      combined = (front_end.rstrip() + ' ' + line.lstrip()).strip()
      write_output(f"Combined w/ front_end: '{combined}'", out_file)

      # note - any straggling illegal actions at the end will just be dropped.

      result = score_it(combined, target, pos2cell, out_file, score, listen_to_environment=False, report_score=False)

  return combined


def grade(output_filename, first_index, test_prefixes, target_lines):
  """
  Given a set of `test_prefixes` representing problems we have to answer, and `target_lines` representing the solutions,
  grade how many of the solutions our model gets correct.

  :param str output_filename: name of output file to write results to
  :param int first_index: index of first problem to grade; may be something other than 0 if we are re-running this code
                          after a GPU crash (we might set it to 73 if it got through problems 0-72 before crashing)
  :param list test_prefixes: list of strings representing problems we have to answer
  :param list target_lines: list of strings representing problem solutions
  :return: None
  """
  print(datetime.datetime.now())
  score = 0
  real_score = 0
  pos2cell = default_pos2cell

  with open(output_filename, 'w', encoding='utf-8') as out_file:
    for i, test_prefix in enumerate(test_prefixes):
      if (i >= first_index):
        write_output(f"== {i} ==", out_file)
        target_line = target_lines[i]
        target = get_result(target_line)
        gpt_line = generate_ef(test_prefix, target, pos2cell, out_file, score, '')

        tries = 1
        while ('\n' not in gpt_line) and tries < MAX_TRIES:
          tries += 1
          write_output('try ' + str(tries), out_file)
          shutil.copyfile(output_filename, OUTPUT_PATH + output_filename)
          tokens = gpt_line.split(' ')
          dividing_word = len(tokens) - MAX_TOKENS
          tail_end = ' '.join(tokens[dividing_word:])
          front_end = ' '.join(tokens[:dividing_word])
          gpt_line = generate_ef(tail_end, target, pos2cell, out_file, score, front_end)

        write_output(f'Final generation for #{i}: {gpt_line}', out_file)
        
        final_string = gpt_line.strip()
        answer = get_result(final_string)
        exactly_correct = (target == answer)
        if exactly_correct:
          real_score += 1

        write_output(f"LOOKING FOR ANSWER WITHIN '{final_string}'...", out_file)
        write_output(f"{exactly_correct}! MODEL ANSWER: {answer}. TARGET ANSWER: {target}", out_file)
        shutil.copyfile(output_filename, OUTPUT_PATH + output_filename)
      write_output(f"SCORE: {real_score}/{len(test_prefixes)}\n", out_file)


def validate_or_test(steps, file_prefix, validation_or_test_file_name, first_index=0):
  """
  Use GPT-Neo to attempt to solve problems from a validation file or test file which 
  ask what the result is after some division operation is completed.

  :param int steps: The number of iterations that the corresponding model ran for.
  :param str file_prefix: A string to place at the beginning of the output file name.
  :param str validation_or_test_file_name: Name of the validation file or test file containing the problems to be solved.
  :param int first_index: index of first problem to attempt; may be something other than 0 if we are re-running this code
                          after a GPU crash (we might set it to 73 if it got through problems 0-72 before crashing)
  :return: None
  """
  grading_suffixes = [str(steps)]
  prefixes = [file_prefix for x in grading_suffixes]
  first_index = first_index

  for file_prefix, grading_suffix in zip(prefixes, grading_suffixes):

    output_filename = 'aitextgen_' + file_prefix + '_graded' + grading_suffix + '.txt'
    print('Outputting to file ' + output_filename + '\n')

    # Load the test prefixes and target answers from file
    with open(validation_or_test_file_name, 'r', encoding='utf-8') as file:
      test_prefixes = []
      sep = '|'
      count = 0
      for line in file:
        parts = line.partition(sep)
        if (count < 4):
          if (len(parts[2]) == 0):
            print(f'Already truncated (nothing after the separator "{sep}")', sep='. ')
          else:
            print(f'Line not already truncated at "{sep}" ({len(parts[2])} characters after it), truncating', sep='. ')
        elif (count == 4):
          print('... (and so on)')
        test_prefixes.append(parts[0] + sep)
        count += 1

    with open(validation_or_test_file_name, 'r', encoding='utf-8') as file:
        target_lines = [line for line in file]

    grade(output_filename, first_index, test_prefixes, target_lines)
    shutil.copyfile(output_filename, OUTPUT_PATH + output_filename)
    first_index = 0

  print(datetime.datetime.now())



## Solve problems in the balanced validation file

In [ ]:
# Arguments:
# steps: The number of iterations that the corresponding model ran for.
# file_prefix: A string to place at the beginning of the output file name.
# validation_or_test_file_name: Name of the validation file or test file containing the problems to be solved. 
# first_index (optional): index of first problem to attempt; may be something other than 0 if we are re-running this code
#                          after a GPU crash (we might set it to 73 if it got through problems 0-72 before crashing)

validate_or_test(10000, 
         file_prefix=f'final_checks', 
         validation_or_test_file_name='val_dataset.txt',
         first_index=0)

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.0"
}



Outputting to file aitextgen_final_checks_graded10000.txt

Line not already truncated at "|" (3819 characters after it), truncating
Line not already truncated at "|" (3935 characters after it), truncating
Line not already truncated at "|" (1448 characters after it), truncating
Line not already truncated at "|" (2626 characters after it), truncating
... (and so on)
2023-05-11 00:47:39.321937
== 0 ==
############ generate_ef ############



Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.0"
}



front_end: 
Prompt (len=268)                     : Calculate 230 5 229 5 228 5 227 1 226 3 225 7 224 3 223 3 222 5 221 7 220 1 219 2 218 1 217 5 216 0 215 8 214 8 213 4 212 7 211 9 210 6 209 2 208 9 207 0 206 7 205 4 204 6 203 2 202 5 201 7 plus 212 1 211 8 210 8 209 2 208 9 207 9 206 2 205 0 204 5 203 1 202 2 201 3 |
Generation (includes prompt) (len=4636): Calculate 230 5 229 5 228 5 227 1 226 3 225 7 224 3 223 3 222 5 221 7 220 1 219 2 218 1 217 5 216 0 215 8 214 8 213 4 212 7 211 9 210 6 209 2 208 9 207 0 206 7 205 4 204 6 203 2 202 5 201 7 plus 212 1 211 8 210 8 209 2 208 9 207 9 206 2 205 0 204 5 203 1 202 2 201 3 | write 71,01:201 7 72,01:202 5 73,01:203 2 74,01:204 6 75,01:205 4 76,01:206 7 77,01:207 0 78,01:208 9 79,01:209 2 80,01:210 6 81,01:211 9 82,01:212 7 83,01:213 4 84,01:214 8 85,01:215 8 86,01:216 0 87,01:217 5 88,01:218 1 89,01:219 2 90,01:220 1 91,01:221 7 92,01:222 5 93,01:223 3 94,01:224 3 95,01:225 7 96,01:226 3 97,01:227 1 98,01:228 5 99,01:229 5 100,01:230 5 { 1

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.0"
}



...

front_end: 
Prompt (len=268)                     : Calculate 225 8 224 1 223 6 222 8 221 4 220 4 219 7 218 1 217 6 216 3 215 1 214 2 213 5 212 3 211 0 210 0 209 2 208 3 207 0 206 5 205 5 204 3 203 6 202 1 201 9 plus 217 7 216 0 215 1 214 8 213 9 212 3 211 1 210 7 209 3 208 9 207 7 206 0 205 8 204 0 203 9 202 0 201 7 |
Generation (includes prompt) (len=4628): Calculate 225 8 224 1 223 6 222 8 221 4 220 4 219 7 218 1 217 6 216 3 215 1 214 2 213 5 212 3 211 0 210 0 209 2 208 3 207 0 206 5 205 5 204 3 203 6 202 1 201 9 plus 217 7 216 0 215 1 214 8 213 9 212 3 211 1 210 7 209 3 208 9 207 7 206 0 205 8 204 0 203 9 202 0 201 7 | write 71,01:201 9 72,01:202 1 73,01:203 6 74,01:204 3 75,01:205 5 76,01:206 5 77,01:207 0 78,01:208 3 79,01:209 2 80,01:210 0 81,01:211 0 82,01:212 3 83,01:213 5 84,01:214 2 85,01:215 1 86,01:216 3 87,01:217 6 88,01:218 1 89,01:219 7 90,01:220 4 91,01:221 4 92,01:222 8 93,01:223 6 94,01:224 1 95,01:225 8 { 17 digits } { add } 71,02:201 7 72,02:202 0 73,02:203 9 74,02:

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.0"
}



front_end: 
Prompt (len=304)                     : Calculate 228 3 227 1 226 1 225 9 224 6 223 9 222 6 221 0 220 7 219 8 218 7 217 2 216 2 215 1 214 3 213 6 212 5 211 7 210 7 209 7 208 8 207 7 206 7 205 8 204 1 203 8 202 3 201 3 plus 220 6 219 2 218 2 217 5 216 6 215 2 214 5 213 8 212 3 211 2 210 2 209 6 208 3 207 6 206 9 205 8 204 3 203 9 202 6 201 7 |
Generation (includes prompt) (len=4678): Calculate 228 3 227 1 226 1 225 9 224 6 223 9 222 6 221 0 220 7 219 8 218 7 217 2 216 2 215 1 214 3 213 6 212 5 211 7 210 7 209 7 208 8 207 7 206 7 205 8 204 1 203 8 202 3 201 3 plus 220 6 219 2 218 2 217 5 216 6 215 2 214 5 213 8 212 3 211 2 210 2 209 6 208 3 207 6 206 9 205 8 204 3 203 9 202 6 201 7 | write 71,01:201 3 72,01:202 3 73,01:203 8 74,01:204 1 75,01:205 8 76,01:206 7 77,01:207 7 78,01:208 8 79,01:209 7 80,01:210 7 81,01:211 7 82,01:212 5 83,01:213 6 84,01:214 3 85,01:215 1 86,01:216 2 87,01:217 2 88,01:218 7 89,01:219 8 90,01:220 7 91,01:221 0 92,01:222 6 93,01:223 9 94,01:224 6 95,0

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.0"
}



front_end: 
Prompt (len=226)                     : Calculate 215 4 214 8 213 3 212 9 211 6 210 9 209 0 208 1 207 3 206 9 205 1 204 6 203 2 202 7 201 3 plus 220 5 219 4 218 6 217 9 216 6 215 2 214 0 213 4 212 1 211 5 210 4 209 0 208 8 207 8 206 6 205 8 204 6 203 3 202 7 201 6 |
Generation (includes prompt) (len=4629): Calculate 215 4 214 8 213 3 212 9 211 6 210 9 209 0 208 1 207 3 206 9 205 1 204 6 203 2 202 7 201 3 plus 220 5 219 4 218 6 217 9 216 6 215 2 214 0 213 4 212 1 211 5 210 4 209 0 208 8 207 8 206 6 205 8 204 6 203 3 202 7 201 6 | write 71,01:201 3 72,01:202 7 73,01:203 2 74,01:204 6 75,01:205 1 76,01:206 9 77,01:207 3 78,01:208 1 79,01:209 0 80,01:210 9 81,01:211 6 82,01:212 9 83,01:213 3 84,01:214 8 85,01:215 4 { 20 digits } { add } 71,02:201 6 72,02:202 7 73,02:203 3 74,02:204 6 75,02:205 8 76,02:206 6 77,02:207 8 78,02:208 8 79,02:209 0 80,02:210 4 81,02:211 5 82,02:212 1 83,02:213 4 84,02:214 0 85,02:215 2 86,02:216 6 87,02:217 9 88,02:218 6 89,02:219 4 90,02:220 5 { 20 d

## Solve problems in the unbalanced validation file (after additional training)

In [21]:
# Arguments:
# steps: The number of iterations that the corresponding model ran for.
# file_prefix: A string to place at the beginning of the output file name.
# validation_or_test_file_name: Name of the validation file or test file containing the problems to be solved. 
# first_index (optional): index of first problem to attempt; may be something other than 0 if we are re-running this code
#                          after a GPU crash (we might set it to 73 if it got through problems 0-72 before crashing)

validate_or_test(10000, 
         file_prefix=f'final_checks', 
         validation_or_test_file_name='val_dataset_unbalanced.txt',
         first_index=0)

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.1"
}



Outputting to file aitextgen_final_checks_graded10000.txt

Line not already truncated at "|" (4467 characters after it), truncating
Line not already truncated at "|" (4516 characters after it), truncating
Line not already truncated at "|" (4376 characters after it), truncating
Line not already truncated at "|" (4384 characters after it), truncating
... (and so on)
2023-05-12 00:27:04.279195
== 0 ==
############ generate_ef ############



Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.1"
}



front_end: 
Prompt (len=370)                     : Calculate 229 8 228 7 227 9 226 3 225 8 224 6 223 5 222 4 221 9 220 3 219 7 218 9 217 0 216 5 215 0 214 2 213 7 212 1 211 6 210 9 209 2 208 6 207 9 206 3 205 9 204 0 203 8 202 4 201 0 plus 230 7 229 3 228 8 227 5 226 3 225 9 224 3 223 2 222 2 221 9 220 5 219 3 218 6 217 2 216 7 215 8 214 8 213 4 212 6 211 5 210 5 209 7 208 8 207 1 206 6 205 6 204 4 203 3 202 3 201 4 |
Generation (includes prompt) (len=4591): Calculate 229 8 228 7 227 9 226 3 225 8 224 6 223 5 222 4 221 9 220 3 219 7 218 9 217 0 216 5 215 0 214 2 213 7 212 1 211 6 210 9 209 2 208 6 207 9 206 3 205 9 204 0 203 8 202 4 201 0 plus 230 7 229 3 228 8 227 5 226 3 225 9 224 3 223 2 222 2 221 9 220 5 219 3 218 6 217 2 216 7 215 8 214 8 213 4 212 6 211 5 210 5 209 7 208 8 207 1 206 6 205 6 204 4 203 3 202 3 201 4 | write 71,01:201 0 72,01:202 4 73,01:203 8 74,01:204 0 75,01:205 9 76,01:206 3 77,01:207 9 78,01:208 6 79,01:209 2 80,01:210 9 81,01:211 6 82,01:212 1 83,01:213 7 84,0

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.1"
}



...

front_end: 
Prompt (len=376)                     : Calculate 230 7 229 7 228 7 227 4 226 9 225 3 224 2 223 3 222 3 221 7 220 1 219 0 218 7 217 9 216 9 215 2 214 8 213 7 212 9 211 5 210 6 209 4 208 6 207 6 206 6 205 7 204 7 203 0 202 9 201 6 plus 230 9 229 8 228 2 227 9 226 7 225 7 224 8 223 8 222 4 221 5 220 4 219 6 218 1 217 0 216 7 215 2 214 7 213 4 212 4 211 1 210 3 209 6 208 5 207 8 206 1 205 5 204 6 203 2 202 3 201 2 |
Generation (includes prompt) (len=4611): Calculate 230 7 229 7 228 7 227 4 226 9 225 3 224 2 223 3 222 3 221 7 220 1 219 0 218 7 217 9 216 9 215 2 214 8 213 7 212 9 211 5 210 6 209 4 208 6 207 6 206 6 205 7 204 7 203 0 202 9 201 6 plus 230 9 229 8 228 2 227 9 226 7 225 7 224 8 223 8 222 4 221 5 220 4 219 6 218 1 217 0 216 7 215 2 214 7 213 4 212 4 211 1 210 3 209 6 208 5 207 8 206 1 205 5 204 6 203 2 202 3 201 2 | write 71,01:201 6 72,01:202 9 73,01:203 0 74,01:204 7 75,01:205 7 76,01:206 6 77,01:207 6 78,01:208 6 79,01:209 4 80,01:210 6 81,01:211 5 82,01:212 9 83,0

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.1"
}



front_end: Calculate 230 7 229 7 228 7 227 4 226 9 225 3 224 2 223 3 222 3 221 7 220 1 219 0 218 7 217 9 216 9 215 2 214 8 213 7 212 9 211 5 210 6 209 4 208 6 207 6 206 6 205 7 204 7 203 0 202 9 201 6 plus 230 9 229 8 228 2 227 9 226 7 225 7 224 8 223 8 222 4 221 5 220 4 219 6 218 1 217 0 216 7 215 2 214 7 213 4 212 4 211 1 210 3 209 6 208 5 207 8 206 1 205 5 204 6 203 2 202 3 201 2 | write 71,01:201 6 72,01:202 9 73,01:203 0 74,01:204 7 75,01:205 7 76,01:206 6 77,01:207 6 78,01:208 6 79,01:209 4 80,01:210 6 81,01:211 5 82,01:212 9 83,01:213 7 84,01:214 8 85,01:215 2 86,01:216 9 87,01:217 9 88,01:218 7 89,01:219 0 90,01:220 1 91,01:221 7 92,01:222 3 93,01:223 3 94,01:224 2 95,01:225 3 96,01:226 9 97,01:227 4 98,01:228 7 99,01:229 7 100,01:230 7 { 30 digits } { add } 71,02:201 2 72,02:202
Prompt (len=3823)                     : 3 73,02:203 2 74,02:204 6 75,02:205 5 76,02:206 1 77,02:207 8 78,02:208 5 79,02:209 6 80,02:210 3 81,02:211 1 82,02:212 4 83,02:213 4 84,02:214 7 85,02:215 2 86,

Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.29.1"
}



front_end: 
Prompt (len=370)                     : Calculate 230 4 229 2 228 6 227 2 226 7 225 7 224 6 223 9 222 3 221 9 220 4 219 4 218 0 217 8 216 5 215 4 214 4 213 1 212 6 211 9 210 9 209 1 208 5 207 8 206 4 205 0 204 4 203 4 202 4 201 6 plus 229 2 228 1 227 8 226 0 225 4 224 2 223 5 222 8 221 9 220 7 219 3 218 4 217 1 216 0 215 9 214 7 213 8 212 2 211 0 210 7 209 1 208 8 207 9 206 5 205 5 204 2 203 7 202 8 201 4 |
Generation (includes prompt) (len=4594): Calculate 230 4 229 2 228 6 227 2 226 7 225 7 224 6 223 9 222 3 221 9 220 4 219 4 218 0 217 8 216 5 215 4 214 4 213 1 212 6 211 9 210 9 209 1 208 5 207 8 206 4 205 0 204 4 203 4 202 4 201 6 plus 229 2 228 1 227 8 226 0 225 4 224 2 223 5 222 8 221 9 220 7 219 3 218 4 217 1 216 0 215 9 214 7 213 8 212 2 211 0 210 7 209 1 208 8 207 9 206 5 205 5 204 2 203 7 202 8 201 4 | write 71,01:201 6 72,01:202 4 73,01:203 4 74,01:204 4 75,01:205 0 76,01:206 4 77,01:207 8 78,01:208 5 79,01:209 1 80,01:210 9 81,01:211 9 82,01:212 6 83,01:213 1 84,0